In [3]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
import random
from icecream import ic
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import logging

logging.basicConfig(level=logging.DEBUG)

In [4]:
CITIES = pd.read_csv('cities/russia.csv', header=None, names=['name', 'lat', 'lon'])

#calcola la distanza tra le citta
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))

#calcola e riempie la matrice delle distanze DIST_MATRIX
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Abakan,53.72,91.43
1,Achinsk,56.28,90.50
2,Almetyevsk,54.90,52.31
3,Angarsk,52.57,103.91
4,Arkhangelsk,64.57,40.53


In [5]:
"""
#computes the cost of a tsp population (the total cost of the routes)
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost
"""

#more optimized function with cache
# Memoization cache for tsp_cost
cost_cache = {}

def tsp_cost(tsp):
    tsp_tuple = tuple(tsp)
    if tsp_tuple in cost_cache:
        return cost_cache[tsp_tuple]
    
    # Calculate the total cost if not in cache
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))
    tot_cost = sum(DIST_MATRIX[c1, c2] for c1, c2 in zip(tsp, tsp[1:]))
    cost_cache[tsp_tuple] = tot_cost
    return tot_cost

In [6]:
# Parameters
#POPULATION_SIZE = 100
#NUM_GENERATIONS = 500
#MUTATION_RATE = 0.2



POPULATION_SIZE = 50    # Increased population size for diversity
NUM_GENERATIONS = 10000   # More generations for improved convergence
MUTATION_RATE = 0.2     # Balanced mutation rate for diversity
NO_IMPROVEMENT_LIMIT = 2000
OFFSPRING_SIZE = 4
NUM_ELITES = max(1, int(POPULATION_SIZE * 0.05))


In [7]:
class Individual:
    def __init__(self, genome):
        self.genome = genome
        self.fitness = -tsp_cost(genome)  # Negative fitness for minimization

# Greedy Nearest Neighbor Initialization
def greedy_nearest_neighbor(start_city=0):
    unvisited = set(range(len(CITIES)))
    unvisited.remove(start_city)
    route = [start_city]
    current_city = start_city
    while unvisited:
        nearest_city = min(unvisited, key=lambda city: DIST_MATRIX[current_city, city])
        route.append(nearest_city)
        unvisited.remove(nearest_city)
        current_city = nearest_city
    route.append(start_city)
    return Individual(route)

# MST-based Initialization
def greedy_mst_based():
    segments = [({c1, c2}, DIST_MATRIX[c1, c2]) for c1, c2 in combinations(range(len(CITIES)), 2)]
    visited = {0}
    edges = set()
    segments.sort(key=lambda e: e[1])
    
    for edge, dist in segments:
        if len(visited) == len(CITIES):
            break
        if edge & visited:
            if not cyclic(edges | {tuple(edge)}):
                edges.add(tuple(edge))
                visited.update(edge)
                
    tsp_route = list(visited) + [list(visited)[0]]
    return Individual(tsp_route)

def cyclic(edges):
    G = nx.Graph()
    G.add_edges_from(edges)
    try:
        nx.find_cycle(G)
        return True
    except:
        return False

In [8]:
# Initialize Population with Greedy Solutions and Random Paths
def initialize_population(size):
    population = [greedy_nearest_neighbor(), greedy_mst_based()]
    for _ in range(size - len(population)):
        random_route = list(range(len(CITIES)))
        random.shuffle(random_route)
        random_route.append(random_route[0])
        population.append(Individual(random_route))
    return population

def adaptive_mutation_rate(generation, mutation_rate, no_improvement_count):
    return mutation_rate * (1 + 0.1 * no_improvement_count)

def multi_point_crossover(p1, p2, points=2):
    # Choose crossover points and ensure p1's genome section is preserved
    points = sorted(random.sample(range(1, len(CITIES) - 1), points))
    child_genome = [None] * len(CITIES)
    child_genome[points[0]:points[1]] = p1.genome[points[0]:points[1]]
    
    # Fill remaining cities from p2 in order, ensuring no duplicates
    p2_index = 1  # Start after the first city
    for i in range(len(CITIES)):
        if child_genome[i] is None:
            # Find the next unused city from p2
            while p2.genome[p2_index] in child_genome:
                p2_index += 1
            child_genome[i] = p2.genome[p2_index]
    
    # Ensure the child returns to the starting city
    child_genome.append(child_genome[0])
    return Individual(child_genome)


# Tournament Selection
def parent_selection(population):
    candidates = sorted(random.sample(population, 3), key=lambda ind: ind.fitness, reverse=True)
    return candidates[0]

# Optimized 2-opt with Early Exit
def two_opt(route):
    best = route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(route) - 2):
            for j in range(i + 2, len(route)):
                new_route = route[:i] + route[i:j][::-1] + route[j:]
                if tsp_cost(new_route + [new_route[0]]) < tsp_cost(best + [best[0]]):
                    best = new_route
                    improved = True
                    break  # Exit after first improvement for faster convergence
            if improved:
                break
    return best



# Sample local search function (2-opt example)
def local_search_2opt(individual):
    # Implementation of a simple 2-opt local search to improve an individual
    # This will go through the tour and attempt to swap segments to reduce total path cost
    best_genome = individual.genome
    best_cost = tsp_cost(best_genome)
    improved = True
    
    while improved:
        improved = False
        for i in range(1, len(best_genome) - 2):
            for j in range(i + 1, len(best_genome) - 1):
                new_genome = best_genome[:]
                new_genome[i:j] = reversed(new_genome[i:j])
                new_cost = tsp_cost(new_genome)
                if new_cost < best_cost:
                    best_genome = new_genome
                    best_cost = new_cost
                    improved = True
                    break
            if improved:
                break

    return Individual(best_genome)

# Order Crossover
def xover(p1, p2):
    start, end = sorted(random.sample(range(1, len(CITIES)), 2))
    child_genome = [None] * len(CITIES)
    child_genome[start:end] = p1.genome[start:end]

    p2_ptr = 0
    for i in range(len(CITIES)):
        if child_genome[i] is None:
            while p2.genome[p2_ptr] in child_genome:
                p2_ptr += 1
            child_genome[i] = p2.genome[p2_ptr]
    child_genome.append(child_genome[0])
    return Individual(child_genome)

In [9]:
#Improved Mutation Strategy with Route Reordering
def mutation(p):
    genome = p.genome.copy()
    i, j, k = sorted(random.sample(range(1, len(genome) - 1), 3))
    new_paths = [
        genome[:i] + genome[i:j][::-1] + genome[j:k] + genome[k:],
        genome[:i] + genome[i:j] + genome[j:k][::-1] + genome[k:],
        genome[:i] + genome[k:j:-1] + genome[i:k] + genome[k:]
    ]
    best_path = min(new_paths, key=tsp_cost)
    return Individual(best_path)

def evolve(population, generations=100_000, mutation_rate=0.3, initial_temperature=10_000.0, cooling_rate=0.995):
    best_path = max(population, key=lambda ind: ind.fitness)
    temperature = initial_temperature
    best_costs = []
    stagnant_generations = 0  # Track stagnation
    
    for generation in range(generations):
        if stagnant_generations > 500:  # Increase mutation rate if stuck
            mutation_rate = min(mutation_rate + 0.05, 0.7)
        
        offspring = []
        
        for _ in range(OFFSPRING_SIZE):
            if random.random() < mutation_rate:
                parent = parent_selection(population)
                child = mutation(parent)
            else:
                parent1 = parent_selection(population)
                parent2 = parent_selection(population)
                child = multi_point_crossover(parent1, parent2, points=2)

            # Apply a simple local search, like 2-opt, on each offspring
            child = local_search_2opt(child)
            offspring.append(child)

        for child in offspring:
            child.fitness = -tsp_cost(child.genome)
            delta_fitness = child.fitness - best_path.fitness

            if delta_fitness > 0 or random.random() < np.exp(delta_fitness / temperature):
                best_path = child if child.fitness > best_path.fitness else best_path
                stagnant_generations = 0  # Reset stagnation if improved
            else:
                stagnant_generations += 1

        # Population management, cooling, and best cost tracking
        population.extend(offspring)
        population.sort(key=lambda ind: ind.fitness, reverse=True)
        population = population[:POPULATION_SIZE]
        temperature *= cooling_rate
        best_costs.append(-best_path.fitness)

    # Plot cost progression
    plt.plot(best_costs, label="Best Cost")
    plt.xlabel("Generation")
    plt.ylabel("Cost")
    plt.title("Cost of Best Path Over Generations")
    plt.legend()
    plt.show()

    return best_path



In [ ]:
# Execute the Evolution
population = initialize_population(10)
best_path = evolve(population)

# Display Results
logging.info(f"Best path found: {best_path.genome}")
logging.info(f"Best path cost: {-best_path.fitness:.2f} km")

In [ ]:
#weird shit more advanced

In [10]:
def edge_assembly_crossover(p1, p2):
    child_genome = []
    # Initialize edges dictionary to count occurrences of edges
    edges = {i: set() for i in range(len(CITIES))}
    
    # Add edges from both parents to each city's edge set
    for genome in [p1.genome, p2.genome]:
        for i in range(len(genome) - 1):
            edges[genome[i]].add(genome[i+1])
            edges[genome[i+1]].add(genome[i])

    # Start with a random city and build path from there
    current_city = random.choice(list(edges.keys()))
    while len(child_genome) < len(CITIES):
        child_genome.append(current_city)
        next_city = min(edges[current_city], key=lambda x: len(edges[x]))  # Choose city with fewest remaining edges
        for connected_city in edges[current_city]:
            edges[connected_city].discard(current_city)
        del edges[current_city]
        current_city = next_city

    # Complete cycle
    child_genome.append(child_genome[0])
    return Individual(child_genome)



def three_opt(individual):
    # Apply 3-opt moves to improve the individual's route
    improved = True
    while improved:
        improved = False
        for i in range(1, len(individual.genome) - 2):
            for j in range(i + 1, len(individual.genome) - 1):
                for k in range(j + 1, len(individual.genome)):
                    new_genome = individual.genome[:]
                    new_genome[i:j] = reversed(new_genome[i:j])
                    new_genome[j:k] = reversed(new_genome[j:k])
                    if tsp_cost(new_genome) < tsp_cost(individual.genome):
                        individual.genome = new_genome
                        improved = True
                        break
    individual.fitness = -tsp_cost(individual.genome)
    return individual




def evolve_smart(population, generations=100_000, mutation_rate=0.3, initial_temperature=10_000.0, cooling_rate=0.995):
    best_path = max(population, key=lambda ind: ind.fitness)
    temperature = initial_temperature

    for generation in range(generations):
        offspring = []

        # Generate offspring through mutation or crossover
        for _ in range(OFFSPRING_SIZE):
            if random.random() < mutation_rate:
                parent = parent_selection(population)
                child = mutation(parent)
            else:
                parent1 = parent_selection(population)
                parent2 = parent_selection(population)
                child = edge_assembly_crossover(parent1, parent2)

            child.fitness = -tsp_cost(child.genome)
            offspring.append(child)

        # Apply 3-opt to top 20% of the new population
        top_offspring = sorted(offspring, key=lambda ind: ind.fitness, reverse=True)[:len(offspring) // 5]
        for ind in top_offspring:
            three_opt(ind)

        # Add offspring to population and keep only the best individuals
        population.extend(offspring)
        population = sorted(population, key=lambda ind: ind.fitness, reverse=True)[:POPULATION_SIZE]

        # Cooling for simulated annealing
        temperature *= cooling_rate

        # Logging every 1,000 generations
        if generation % 1_000 == 0:
            completion_percentage = (generation / generations) * 100
            logging.info(f"Generation {generation} ({completion_percentage:.2f}% completed): Best cost {-best_path.fitness:.2f} km")

    logging.info("Evolution complete.")
    return best_path



In [11]:
population = initialize_population(10)
best_path = evolve_smart(population)

# Display Results
logging.info(f"Best path found: {best_path.genome}")
logging.info(f"Best path cost: {-best_path.fitness:.2f} km")

ValueError: min() iterable argument is empty